In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.webdriver import WebDriver as ChromeWebDriver
from selenium.common.exceptions import NoSuchElementException
import csv
import time
import random
import os

In [5]:
options = Options()
# options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-notifications")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-translate")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
options.add_argument("--window-size=1920x1080")
options.add_argument("--remote-debugging-port=9222")
options.add_argument("--disable-web-security")
options.add_argument("--allow-running-insecure-content")
options.add_argument("--ignore-certificate-errors")
options.add_argument("--incognito")
options.add_argument("--lang=en-US")


In [ ]:
chrome_service = Service(r"C:\Users\KUNAL VERMA\.wdm\drivers\chromedriver\win64\133.0.6943.141\chromedriver-win32\chromedriver.exe") 
# Update with your chromedriver path


In [7]:
driver =  webdriver.Chrome(service=chrome_service, options=options)

In [11]:
driver.get("https://ca.trustpilot.com/review/snowbirdadvisorinsurance.ca")  # Open Indeed homepage

In [18]:
def close_popup(driver):
    try:
        # Wait for the popup to appear (max 5 seconds)
        popup_div = driver.find_element(By.XPATH, '//*[@id="onetrust-banner-sdk"]')

        # Find and click the "Got it" button
        got_it_button = popup_div.find_element(By.ID, "onetrust-accept-btn-handler")
        got_it_button.click()
        print("✅ Popup closed successfully!")

    except NoSuchElementException:
        print("⚠️ No popup found.")
    except Exception as e:
        print(f"❌ Error while closing popup: {e}")

# Call this function after opening the website
close_popup(driver)

✅ Popup closed successfully!


In [19]:
title_div = driver.find_element(By.ID, 'business-unit-title')
title = title_div.find_element(By.TAG_NAME, 'h1').text
print("Title:", title)

Title: Snowbird Advisor Insurance 
Reviews 5,366


In [20]:
more_filter = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div/div/main/div/div[2]/div[2]/div[3]/div/div/div[2]/section[1]/div[1]/button')
print(more_filter.text)  # Click on the "Write a review" button
# print(more_filter.get_dom_attribute('parentElement'))
# print(more_filter.get_attribute('innerHTML'))  # Click on the "Write a review" button
more_filter.click()
time.sleep(5)

More filters


In [9]:
parent_varified = driver.find_element(By.XPATH, '//*[@id="filter-verified"]')
print(parent_varified.text)  # Click on the "Write a review" button
parent_varified.click()

In [10]:
show_results = driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div/div[3]/div/button[2]')
print(show_results.text)  # Click on the "Write a review" button
show_results.click()

Show 5,220 reviews


In [ ]:


# 🔹 Create CSV File and Write Header (Append Mode)
csv_file = open("reviews3.csv", "a", newline="", encoding="utf-8")
csv_writer = csv.writer(csv_file)

# 🔹 Define Human-Like Delay Function
def human_delay(min_delay=random.randint(1,2), max_delay=random.randint(2,3)):
    """Pauses execution for a random duration between min_delay and max_delay seconds."""
    delay = random.randint(min_delay, max_delay)
    # print(f"⏳ Waiting {delay} seconds to mimic human behavior...")
    time.sleep(delay)

# 🔹 Function to Scroll Inside a Scrollable Div
def scroll_inside_reviews():
    review_container = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div/div/main/div/div[2]/div[2]/div[3]/div/div/div[2]/section[2]/div[1]')  # Adjust class if needed
    last_height = driver.execute_script("return arguments[0].scrollHeight;", review_container)
    
    while True:
        driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", review_container)
        human_delay(1, 2)  # Wait for content to load
        new_height = driver.execute_script("return arguments[0].scrollHeight;", review_container)
        
        if new_height == last_height:  # Stop when no new content is loaded
            break
        last_height = new_height

# 🔹 Loop Until No More Pages
for i in range(1,101):
    driver.get(f"https://ca.trustpilot.com/review/snowbirdadvisorinsurance.ca?page={i}&verified=true")

    # 🔹 Scroll Inside the Reviews Section
    scroll_inside_reviews()

    # 🔹 Find All Review Cards
    cards = driver.find_elements(By.CLASS_NAME, "styles_cardWrapper__kOLEb")
    # print(f"✅ Found {len(cards)} reviews on this page")

    for card in cards:
        try:
            driver.execute_script("arguments[0].scrollIntoView();", card)  # Scroll to card
            human_delay(1, 3)  # Random delay before extracting data

            review_content = card.find_element(By.CLASS_NAME, "styles_reviewContent__SCYfD")
            heading = review_content.find_element(By.TAG_NAME, 'a').text.strip()
            review_text_elements = review_content.find_elements(By.TAG_NAME, 'p')
            review_text = review_text_elements[0].text.strip() if review_text_elements else ""

            # 🔹 Write Data to CSV
            csv_writer.writerow([heading, review_text])
            # print(f"📌 Saved: {heading[:30]}...")

            human_delay(2, 3)  # Random delay after extracting data

        except Exception as e:
            print(f"❌ Skipping a review due to error: {e}")

    
csv_file.close()
driver.quit()
print("✅ All reviews saved to 'reviews.csv' successfully!")
